In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1843524/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_7,0.995317,0.743234,0.966706,0.984333,0.980867,0.016834,1.716996,0.028836,0.050046,0.039441,0.482169,0.129748,1.017355,0.131824,168.168659,340.879306,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_9_6,0.995317,0.743148,0.966738,0.984507,0.981008,0.016835,1.717573,0.028809,0.049493,0.039151,0.486170,0.129751,1.017356,0.131827,168.168542,340.879189,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_9_8,0.995316,0.743319,0.966665,0.984161,0.980724,0.016836,1.716429,0.028872,0.050599,0.039736,0.478186,0.129755,1.017357,0.131831,168.168443,340.879090,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_9_5,0.995315,0.743060,0.966761,0.984680,0.981147,0.016839,1.718162,0.028788,0.048938,0.038863,0.490198,0.129766,1.017360,0.131843,168.168075,340.878721,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_9_9,0.995315,0.743402,0.966614,0.983988,0.980580,0.016841,1.715874,0.028916,0.051150,0.040033,0.474250,0.129773,1.017362,0.131849,168.167877,340.878524,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,model_1_1_4,0.872092,0.603730,0.835401,0.878057,0.860533,0.459783,2.649859,0.769441,0.804782,0.787112,1.119391,0.678073,1.474013,0.688923,161.554001,334.264647,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
415,model_1_1_3,0.869839,0.601934,0.833762,0.875627,0.858433,0.467880,2.661869,0.777107,0.820821,0.798964,1.130775,0.684018,1.482361,0.694963,161.519085,334.229732,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
416,model_1_1_2,0.867540,0.600108,0.832057,0.873125,0.856264,0.476145,2.674077,0.785076,0.837332,0.811204,1.141726,0.690033,1.490881,0.701074,161.484065,334.194712,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
417,model_1_1_1,0.865186,0.598257,0.830321,0.870546,0.854037,0.484607,2.686457,0.793190,0.854354,0.823772,1.152936,0.696137,1.499605,0.707276,161.448835,334.159482,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
